In [ ]:
import pandas as pd
df1 = pd.read_csv('BTC_Trial_NB.csv')
df1.columns

In [ ]:
df1.index = pd.to_datetime(df1['Date'], format='%d/%m/%Y')
df1.head()

In [50]:
df = df1.drop(['Date', 'Network Difficulty'],axis=1)

In [ ]:
df.shape

In [ ]:
import pandas as pd

# columns_to_lag_1_7 = ['VOL1','SP500','Interest Rate', 'RUSSELL']
columns_to_lag_1_7 = ['VOL1', 'ClosePrice ', 'OpenPrice', 'HighPrice', 'LowPrice','Market Volumn']
columns_to_lag_1_5 = [
    'Average Block Size', 'Average Transaction Per Block', 'Hash Rate', 'Transactions Per Second','Oil', 'Gold', 'Silver', 'USDEUR', 'USDYUAN',
    'Economic Uncertainty', 'Crypto Index']

# Create lagged features for 1-7 lags for 'VOL1' and 'ClosePrice'
for column in columns_to_lag_1_7:
    for lag in range(1, 7):
        df[f'{column}_lag{lag}'] = df[column].shift(lag)

# Create lagged features for 1-2 lags for the rest of the columns
for column in columns_to_lag_1_5:
    for lag in range(1, 7):
        df[f'{column}_lag{lag}'] = df[column].shift(lag)

# Include the original 'VOL1' and 'ClosePrice' columns
new_df = df[['VOL1', 'ClosePrice ','Month', 'Year'] + [f'{col}_lag{lag}' for col in columns_to_lag_1_7 for lag in range(1, 7)] + [f'{col}_lag{lag}' for col in columns_to_lag_1_5 for lag in range(1, 7)]]

# Drop rows with NaN values created due to lagging
new_df = new_df.dropna()

In [ ]:
new_df.columns

In [54]:
x = new_df.drop(columns = ['ClosePrice ','VOL1'])
y = new_df['ClosePrice ']

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=False)

In [56]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled=scaler.transform(X_test)

X_train_scaled = X_train
X_test_scaled=X_test

In [ ]:
X_train_scaled.shape, X_test_scaled.shape

In [58]:
from sklearn.ensemble import RandomForestRegressor

In [59]:
rf = RandomForestRegressor(
    n_estimators=300,        # number of trees in the forest
    max_depth= None,            # maximum depth of the tree
    min_samples_split=10,     # minimum number of samples required to split an internal node
    max_features=0.5,     # number of features to consider when looking for the best split
    random_state=24          # random seed for reproducibility
)
# Fit the model on the training data
rf.fit(X_train_scaled, y_train)

# Predict on the test data
y_pred = rf.predict(X_test_scaled)

In [60]:
# RMSE
from sklearn.metrics import mean_squared_error
import numpy as np
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# MAPE
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#NMSE
def nmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    variance = np.var(y_true)
    return mse / variance

#DA
def DA(y_true, y_pred):
    # Convert the arrays to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate the direction of change
    true_direction = np.sign(np.diff(y_true))
    pred_direction = np.sign(np.diff(y_pred))
    
    # Compare directions
    correct_direction = np.sum(true_direction == pred_direction)
    total_direction = len(true_direction)
    
    # Calculate directional accuracy
    da = correct_direction / total_direction * 100
    
    return da

In [ ]:
y_true = y_test
y_predi = y_pred

print("RMSE: ", rmse(y_true, y_predi))
print("MAPE: ", mape(y_true, y_predi))
print("NMSE: ", nmse(y_true, y_predi))
print("DA: ", DA(y_true, y_predi))

In [ ]:
# prediction_df = pd.DataFrame(y_pred, columns=['Predictions'])
# prediction_df.index = y_test.index

p=700

import matplotlib.pyplot as plt
import numpy as np
# Plotting
plt.figure(figsize=(12, 4))

plt.plot(y_test.index[-p:], y_test[-p:], label='Actual Price', color='b')
plt.plot(y_test.index[-p:], y_pred[-p:], label='Predicted Price', color='r', alpha=0.7)
plt.title('Actual vs Predicted Price (BTC)')
plt.legend()
plt.grid(True)
plt.show()

In [63]:
y_pred = pd.DataFrame(y_pred, columns=['RF'])
y_pred.to_csv('RF.csv')

In [ ]:
from sklearn import metrics

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error (RMSE):', metrics.mean_squared_error(y_test, y_pred, squared=False))
print('Mean Absolute Percentage Error (MAPE):', metrics.mean_absolute_percentage_error(y_test, y_pred))
print('Explained Variance Score:', metrics.explained_variance_score(y_test, y_pred))
print('Max Error:', metrics.max_error(y_test, y_pred))
print('Mean Squared Log Error:', metrics.mean_squared_log_error(y_test, y_pred))
print('Median Absolute Error:', metrics.median_absolute_error(y_test, y_pred))
print('R^2:', metrics.r2_score(y_test, y_pred))
print('Mean Poisson Deviance:', metrics.mean_poisson_deviance(y_test, y_pred))
print('Mean Gamma Deviance:', metrics.mean_gamma_deviance(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import pandas as pd

# Get feature importances
importances = rf.feature_importances_

# Create a DataFrame for feature importances
feature_names = x.columns
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

top_features_df = feature_importances_df.head(10)

# Plot top 5 feature importances
plt.figure(figsize=(12, 4))
plt.barh(top_features_df['Feature'], top_features_df['Importance'])
plt.xlabel('Importance')
plt.title('Top 10 Feature Importances')
plt.gca().invert_yaxis()
plt.show()